# Importazione librerie

In [1]:
import re
import spacy
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
print("Importazione librerie completata")

Importazione librerie completata


# Pipeline articolo di giornale

In [9]:
class PipelineNotizia:
    def __init__(self, lingua='italian'):
        """Pipeline NLP per articoli di notizie"""
        self.lingua = lingua
        self.nlp = spacy.load('it_core_news_sm' if lingua == 'italian' else 'en_core_web_sm')
        self.stemmer = SnowballStemmer(lingua)
        self.stop_words = set(stopwords.words(lingua))
        
    def normalizza_testo(self, testo):
        """Normalizzazione leggera: rimuove punteggiatura ma preserva maiuscole"""
        testo = re.sub(r'[^\w\sàèéìòù]', ' ', testo)  # Rimuove punteggiatura
        testo = re.sub(r'\s+', ' ', testo).strip()
        return testo
    
    def tokenizza_e_lemmatizza(self, testo):
        """Tokenizza e lemmatizza"""
        doc = self.nlp(testo)
        tokens = []
        for token in doc:
            if not token.is_punct and not token.is_space and token.is_alpha:
                lemma = token.lemma_ if token.lemma_ != '-PRON-' else token.text
                tokens.append((token.text, lemma, token.pos_))
        return tokens
    
    def estrai_entita(self, testo):
        """Estrae Named Entities dal testo (es. organizzazioni, luoghi, ecc.)"""
        doc = self.nlp(testo)
        entita = [ent.text for ent in doc.ents]
        return entita
    
    def rimuovi_stop_words(self, tokens):
        """Rimuove stopwords, mantenendo solo parole chiave"""
        return [lemma for (text, lemma, pos) in tokens if lemma.lower() not in self.stop_words]
    
    def filtra_per_pos(self, tokens):
        """Mantiene solo sostantivi e verbi principali"""
        return [lemma for (text, lemma, pos) in tokens if pos in ['NOUN', 'VERB']]
    
    def processa_testo(self, testo, usa_ner=True):
        """Pipeline completa"""
        testo_norm = self.normalizza_testo(testo)
        tokens = self.tokenizza_e_lemmatizza(testo_norm)
        
        # Rimozione stopwords e filtraggio per parte del discorso
        tokens_filtrati = self.rimuovi_stop_words(tokens)
        tokens_filtrati = [lemma for (text, lemma, pos) in tokens if lemma in tokens_filtrati and pos in ['NOUN', 'VERB']]
        
        return {
            'tokens_chiave': list(set(tokens_filtrati)),
        }

# Test

In [10]:
# Test 3 — Articolo di Notizia
testo3 = "Il governo ha annunciato oggi un nuovo piano di incentivi per sostenere le piccole e medie imprese nel settore tecnologico."

pipeline_news = PipelineNotizia()
risultato3 = pipeline_news.processa_testo(testo3)

print("📰 Tokens chiave:", risultato3['tokens_chiave'])

📰 Tokens chiave: ['impresa', 'incentivo', 'sostenere', 'annunciare', 'governo', 'settore', 'piano']


# Approccio utilizzato
Per un articolo di notizia, l’obiettivo è isolare le informazioni chiave (soggetti, azioni, oggetti) e riconoscere entità (istituzioni, aziende, ecc.).
Quindi la pipeline deve:
- mantenere il testo pulito ma senza perdere i nomi propri;
- filtrare i sostantivi e verbi principali, perché in un articolo sono i portatori di significato.